# Pytorch
---------

### Author Information
**Author:** PJ Gibson  
**Email:** Peter.Gibson@doh.wa.gov  
**Github:**   https://github.com/DOH-PJG1303

### Project Information
**Created Date:** 2023-08-09  
**Last Updated:** 2023-08-09  
**Version:** 1  

### Description

In this notebook, we train a ML model for record linkage.
We'll use PyTorch.



### Notes

*\*If you are unfamiliar with the origins of this synthetic data, please see the [Synthetic-Gold](https://github.com/DOH-PJG1303/Synthetic-Gold) github project. We ran the simulation for the state of Nebraska, so all data is relevant to that state.
To manage the size of the data we'll have publicly stored on Github, we only captured relevant data for each table for the population living in years 2019-2022*


*\*\*Annotation improved with the help of chat-GPT*

## 1. Import Libraries

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam, SGD, RMSprop

import mlflow
import mlflow.pytorch
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
import time

In [2]:
# Set random seed for code reproducibility
random.seed(42)
torch.manual_seed(42)

## 2. Prep Data

### 2.1 Read Data

In [3]:
df = pd.read_parquet('../../Data/Training/04. Training Data.parquet')

### 2.2 Test Train Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('label',axis=1), df['label'], test_size=0.2, random_state=42)

## 3. TensorFlow

### 3.1 Conversion

Can't have it resting as a pandas dataframe.
Needs to be tensor.

In [5]:
# Convert pandas DataFrames to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values.astype('float32'))
y_train_tensor = torch.tensor(y_train.values.astype('float32'))
X_test_tensor = torch.tensor(X_test.values.astype('float32'))
y_test_tensor = torch.tensor(y_test.values.astype('float32'))

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders (you can adjust the batch_size and shuffle as needed)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### 3.2 Define Param Grid Search Space


In [6]:
param_grid = [
    [(64,), (100, 50), (30, 20, 10), (50, 50, 50), (10, 10, 10, 10)], # Hidden dimensions
    [0.1, 0.01, 0.001],                                               # Learning rates
    [5, 10, 20],                                                      # Number of epochs
    ['Adam', 'SGD', 'RMSprop']                                        # Optimizers
]

### 3.3 Model Wrapping/Defining

In [7]:
class PyTorchModelWrapper(mlflow.pyfunc.PythonModel):
    """A wrapper class for PyTorch models that implements the mlflow.pyfunc.PythonModel interface."""
    
    def __init__(self, model):
        self.model = model
        
    def predict(self, context, model_input):
        with torch.no_grad():
            model_input_tensor = torch.tensor(model_input.values, dtype=torch.float32)
            output = self.model(model_input_tensor)
            return output.numpy()[:, 0]

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(MLP, self).__init__()
        layers = []
        previous_dim = input_dim
        for dim in hidden_dims:
            layers.extend([
                nn.Linear(previous_dim, dim),
                nn.ReLU()
            ])
            previous_dim = dim
        layers.append(nn.Linear(previous_dim, 1))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

## 4 MLFlow config

### 4.1 Create Experiment

In [8]:
# Define experiment name
experiment_name = 'Record Linkage ML Model Training'

try:
    mlflow.create_experiment(experiment_name)
except:
    pass

# Set the experiment ID
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///c:/Users/pjg1303/Documents2/Python-Record-Linkage/Scripts/Train%20Model/mlruns/395511990306693902', creation_time=1692032458330, experiment_id='395511990306693902', last_update_time=1692032458330, lifecycle_stage='active', name='Record Linkage ML Model Training', tags={}>

### 4.2 Define Run

In [10]:
# Naming it Iter1 (Iteration 1), 12 linking fields (to capture number raw data fields required)
# [fname, mname, lname, dob, sex, add, zip, city, county, state, phone, email]
run_name = 'Iter1, 12 linking fields'

### 4.3 Log Metrics Function

In [11]:
def log_metrics(y_test, pred_proba_test):
    """Log various performance metrics."""

    test_auc_score = roc_auc_score(y_test, pred_proba_test)
    mlflow.log_metric('auc', test_auc_score)
    predict_binary_test = (pred_proba_test > 0.5).astype(int)
    test_f1_score = f1_score(y_test, predict_binary_test)
    mlflow.log_metric('f1', test_f1_score)
    test_accuracy = accuracy_score(y_test, predict_binary_test)
    mlflow.log_metric('accuracy', test_accuracy)
    test_precision = precision_score(y_test, predict_binary_test)
    mlflow.log_metric('precision', test_precision)
    test_recall = recall_score(y_test, predict_binary_test)
    mlflow.log_metric('recall', test_recall)

### 4.4 Set Optimizer Function

Mostly here for readability when called on by perform_run() function

In [12]:
def select_optimizer(optimizer_name, parameters, lr):
    optimizers = {
        'Adam': lambda: Adam(parameters, lr=lr),
        'SGD': lambda: SGD(parameters, lr=lr),
        'RMSprop': lambda: RMSprop(parameters, lr=lr)
    }
    optimizer_func = optimizers.get(optimizer_name)
    if optimizer_func is None:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")
    return optimizer_func()

## 5. Experiment

In [15]:
def perform_run(run_name, param_grid, n_runs, train_loader, test_loader):
    input_size = train_loader.dataset.tensors[0].size(1)
    mlflow.pytorch.autolog()

    for i in range(n_runs):
        hidden_dims, lr, epochs, optimizer_name = [random.choice(param) for param in param_grid]

        with mlflow.start_run(run_name=run_name):
            mlflow.log_param('Model Type', 'Pytorch-MLPClassifier')
            mlflow.log_param('learning_rate', lr)
            mlflow.log_param('hidden_layer_sizes', str(hidden_dims))
            mlflow.log_param('optimizer', optimizer_name)
            mlflow.log_param('epochs', epochs)

            model = MLP(input_size, hidden_dims)
            criterion = nn.BCEWithLogitsLoss()
            optimizer = select_optimizer(optimizer_name, model.parameters(), lr)

            # Training
            start_fit_time = time.time()
            for epoch in range(epochs):
                for batch_X, batch_y in train_loader:
                    optimizer.zero_grad()
                    outputs = model(batch_X)
                    loss = criterion(outputs.squeeze(), batch_y)
                    loss.backward()
                    optimizer.step()
            fit_time = time.time() - start_fit_time
            mlflow.log_metric('fit_time', fit_time) # Logging training time

            # Testing and logging metrics
            start_predict_time = time.time()
            pred_proba_test = []
            with torch.no_grad():
                for batch_X, _ in test_loader:
                    outputs = model(batch_X)
                    pred_proba_test.extend(torch.sigmoid(outputs.squeeze()).numpy())
            predict_time = time.time() - start_predict_time
            mlflow.log_metric('predict_time', predict_time) # Logging prediction time

            pred_proba_test = np.array(pred_proba_test)
            log_metrics(test_loader.dataset.tensors[1].numpy(), pred_proba_test)

            # Logging model to MLflow
            mlflow.pytorch.log_model(model, "model")

In [15]:
# Define how many runs we want.  Do not exceed number of combinations of param_grid for repetition's sake
n_runs = 50

# Perform the runs!
perform_run(run_name, param_grid, n_runs, train_loader, test_loader)

2023/08/15 16:30:26 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
c:\Users\pjg1303\Documents2\Python-Record-Linkage\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\pjg1303\Documents2\Python-Record-Linkage\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\pjg1303\Documents2\Python-Record-Linkage\.venv\lib\site-packages\sklearn\metrics\_classification.py:146

### 4. Inspect Output

Open a CMD terminal.  Navigate to this directory. If you're starting in the root repo folder, this will look like:

```cmd
cd "Scripts\Train Model"
```

Then navigate into the MLFlow UI by typing:

```cmd
mlflow ui
```

There you can interact with models, compare different models, and select one to register.

In [16]:
run_name2 = 'Iter2, 12 linking fields'

param_grid2 = [
    [(200,), (100, 100, 100), (50, 50, 50), (10, 10, 10, 10, 10)], # Hidden dimensions
    [0.005, 0.001, 0.0005],                                               # Learning rates
    [5, 10, 20],                                                      # Number of epochs
    ['Adam', 'RMSprop']                                        # Optimizers
]

In [17]:
# Define how many runs we want.  Do not exceed number of combinations of param_grid for repetition's sake
n_runs = 50

# Perform the runs!
perform_run(run_name2, param_grid2, n_runs, train_loader, test_loader)

2023/08/16 14:27:50 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
c:\Users\pjg1303\Documents2\Python-Record-Linkage\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
